In [1]:
import numpy as np
import pandas as pd
import itertools as it
import pdb

In [2]:
%load_ext line_profiler

In [3]:
df_A = pd.read_excel("Deudas.xlsx")
df_A

,Documento,Nombre,Deuda
0,101,Alice,21
1,102,Bob,123
2,103,Alice,84
3,104,Carol,49


In [4]:
df_B = pd.read_excel("Pagos.xlsx")
df_B

,Documento,Nombre,Pago
0,201,Bob,59
1,202,Alice,49
2,203,Carol,21
3,204,Alice,21
4,205,Bob,64
5,206,Bob,27
6,207,Alice,14
7,208,Carol,12
8,209,Alice,15
9,210,Carol,37


## Uno-a-muchos

In [5]:
%timeit df_B.groupby("Nombre")["Documento"].apply(lambda x: x.values)

593 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
def obtener_documentos_por_nombre(nombres, documentos, pagos):
    # nombres_unicos = [Alice Bob Carol]
    n_uni = np.unique(nombres)
    
    i_ord = np.argsort(nombres)
    # nombres_ordenados = [Alice Alice Alice Alice || Bob Bob Bob || Carol Carol Carol Carol]
    n_ord = nombres[i_ord]
    # documentos_ordenados = [202 204 207 209 || 201 205 206 || 203 208 210 211]
    d_ord = documentos[i_ord]
    # pagos_ordenados = [49 21 14 15 || 59 64 27 || 21 12 37 28]
    p_ord = pagos[i_ord]
    
    # Buscar los límites de los grupos, por ejemplo, dónde deja de ser Alice para pasar a ser Bob, etc
    #  |A A A A|B B B|C C C C|
    # [1 0 0 0 1 0 0 1 0 0 0 1]
    # lim = [0 4 7 11]
    lim = np.where(np.append([""], n_ord) != np.append(n_ord, [""]))[0]
    
    diccionario = {nombre: np.array([d_ord[lim[i]:lim[i+1]], p_ord[lim[i]:lim[i+1]]]) for i, nombre in enumerate(n_uni)}
    
    return diccionario

In [7]:
obtener_documentos_por_nombre(df_B["Nombre"].values, df_B["Documento"].values, df_B["Pago"].values)

{'Alice': array([[202, 204, 207, 209],
        [ 49,  21,  14,  15]], dtype=int64),
 'Bob': array([[201, 205, 206],
        [ 59,  64,  27]], dtype=int64),
 'Carol': array([[203, 208, 210, 211],
        [ 21,  12,  37,  28]], dtype=int64)}

In [10]:
def agrupar(ids3, doc3, mon3, ids5, doc5, mon5):
    ids_uni = np.intersect1d(ids3, ids5)
    
    # Ordenar IDs, documentos y montos en base a las IDs
    ind3_ord = np.argsort(ids3)
    ids3_ord = ids3[ind3_ord]
    doc3_ord = doc3[ind3_ord]
    mon3_ord = mon3[ind3_ord]
    
    ind5_ord = np.argsort(ids5)
    ids5_ord = ids5[ind5_ord]
    doc5_ord = doc5[ind5_ord]
    mon5_ord = mon5[ind5_ord]
    
    # Registrar los cambios de grupos, es decir, cuándo un elemento ya no es igual al siguiente
    # Para eso, comparar el arreglo de IDs con una copia desplazada de sí mismo
    lim3 = np.where(np.append([""], ids3_ord) != np.append(ids3_ord, [""]))[0]
    lim5 = np.where(np.append([""], ids5_ord) != np.append(ids5_ord, [""]))[0]
    
    # Obtener los grupos
    n_ids = len(ids_uni)
    lista = [
        (
            doc3_ord[lim3[i] : lim3[i+1]],
            mon3_ord[lim3[i] : lim3[i+1]],
            doc5_ord[lim5[i] : lim5[i+1]],
            mon5_ord[lim5[i] : lim5[i+1]]
        ) for i in range(n_ids)
    ]
    
    # return ids_uni, doc3_grp, mon3_grp, doc5_grp, mon5_grp
    return ids_uni, *zip(*lista)

In [11]:
ids3 = df_A["Nombre"].values
doc3 = df_A["Documento"].values
mon3 = df_A["Deuda"].values

ids5 = df_B["Nombre"].values
doc5 = df_B["Documento"].values
mon5 = df_B["Pago"].values

agrupar(ids3, doc3, mon3, ids5, doc5, mon5)

(array(['Alice', 'Bob', 'Carol'], dtype=object),
 (array([101, 103], dtype=int64),
  array([102], dtype=int64),
  array([104], dtype=int64)),
 (array([21, 84], dtype=int64),
  array([123], dtype=int64),
  array([49], dtype=int64)),
 (array([202, 204, 207, 209], dtype=int64),
  array([201, 205, 206], dtype=int64),
  array([203, 208, 210, 211], dtype=int64)),
 (array([49, 21, 14, 15], dtype=int64),
  array([59, 64, 27], dtype=int64),
  array([21, 12, 37, 28], dtype=int64)))